In [11]:
# importing libraries
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup

In [12]:
base_url = "https://www.worldfootball.net/schedule/eng-premier-league-2018-2019-spieltag/"

In [13]:
def parse_page(link):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [14]:
def get_team_names(tables):
    names = []
    for team in tables[1].findAll('a'):
        names.append(team.text)
    return names

In [15]:
def get_points(tables):
    points = []
    names = [get_team_names(tables)]
    
    for team in tables[1].findAll('tr'):
        count = 0
        # process each record in row at a time
        for rec in team.findAll('td'):
            # only interested in 10th record (points)
            if (count == 9):
                points.append(rec.text)
            count += 1
    
    names.append(points)
    standing = pd.DataFrame.from_records(names).T
    standing.columns = ["TeamName", "Week"]
    return standing

In [16]:
# for each matchweek we need to scrape a page
cnt = 0
for matchweek in range(1, 39):
    # concantenate strings to create url
    url = base_url + str(matchweek)
    
    # scrape a webpage
    soup = parse_page(url)
    
    # select all tables from page (there are 2 tables on page)
    data = soup.select('table.standard_tabelle')
    
    # select all 'a' tags (Team Names) from second table (point table)
    if cnt == 0:
        TeamNames = get_team_names(data)
        weekly_points = pd.DataFrame(TeamNames)
        weekly_points.columns = ["TeamName"]
        cnt += 1
    
    # process one row at a time in table to get points 
    points = get_points(data)   
    weekly_points = weekly_points.merge(points, on = 'TeamName', suffixes=('',str(matchweek)))
    


In [17]:
weekly_points.head()

,TeamName,Week,Week2,Week3,Week4,Week5,Week6,Week7,Week8,Week9,...,Week29,Week30,Week31,Week32,Week33,Week34,Week35,Week36,Week37,Week38
0,Liverpool FC,3,6,9,12,15,18,19,20,23,...,70,73,76,79,82,85,88,91,94,97
1,Chelsea FC,3,6,9,12,15,16,17,20,21,...,59,60,60,63,66,66,67,68,71,72
2,AFC Bournemouth,3,6,7,7,10,10,13,16,17,...,34,37,38,38,38,41,41,42,45,45
3,Crystal Palace,3,3,3,3,6,7,7,7,7,...,33,33,33,36,39,39,42,43,46,49
4,Manchester City,3,6,7,10,13,16,19,20,23,...,71,74,77,80,83,86,89,92,95,98


In [18]:
# exporting file in csv format
weekly_points.to_csv(path_or_buf= "weekly_points.csv",index=False)